In [ ]:
#https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0' # Set to -1 if CPU should be used CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
elif cpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2
# pip install imutils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
# <Option 1> You need to import GlobalAveragePooling2D if you want to use GAP model
#from keras.layers import GlobalAveragePooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [ ]:
class StridedNet:
	@staticmethod
	def build(width, height, depth, classes, reg, init="he_normal"):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# our first CONV layer will learn a total of 16 filters, each
		# Of which are 7x7 -- we'll then apply 2x2 strides to reduce
		# the spatial dimensions of the volume
		model.add(Conv2D(16, (7, 7), strides=(2, 2), padding="valid", kernel_initializer=init, kernel_regularizer=reg, input_shape=inputShape))

		# here we stack two CONV layers on top of each other where
		# each layerswill learn a total of 32 (3x3) filters
		#NK: <Option 2> You SHOULD design your own CNN models based on StrideNet (old) model
		#NK: Five components of Convolution layer: Conv2D, BatchNormalization, Maxpooling2D, Dropout, Flatten or GlobalAveragePooling2D
		model.add(Conv2D(32, (3, 3), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		#NK: <Option 3> Conv2, same padding, stride (2,2) is essentially same as MaxPooling2D
		#NK: You can change following lines to MaxPooling2D
		model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Dropout(0.25))
		#NK: stack two more CONV layers, keeping the size of each filter
		#NK: as 3x3 but increasing to 64 total learned filters
		model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		#NK: <Option 3> Conv2, same padding, stride (2,2) is essentially same as MaxPooling2D
		#NK: You can change following lines to MaxPooling2D
		model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Dropout(0.25))

		# increase the number of filters again, this time to 128
		model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		#NK: <Option 3> Conv2, same padding, stride (2,2) is essentially same as MaxPooling2D
		#NK: You can change following lines to MaxPooling2D
		model.add(Conv2D(128, (3, 3), strides=(2, 2), padding="same", kernel_initializer=init, kernel_regularizer=reg))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Dropout(0.25))

		#NK: <Option 4> You can convert Flatten into GlobalAveragePooling2D, but you have to make sure that you have enough numbers of filters
		#NK: If the number of filter is small, performance of GlobalAveragePooling2D is not that good
		# fully-connected layer
		model.add(Flatten())
		#NK: <Option 5> You can add FC (fully connected) layers, up to 2 times
		#NK: 3 or more FC layers are not recommended due to long calculation time and increase of parameters
		model.add(Dense(512, kernel_initializer=init))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		#NK: <Option 6> You can change ratio of Dropout depending on how large is the image space you need in CNN model
		model.add(Dropout(0.5))

		#NK: <Caveat> DO NOT CHANGE THIS PART, FINAL LAYER SHOULD BE Dense(#class) with softmax
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
# initialize the set of labels from the CALTECH-101 dataset we are
# going to train our network on
LABELS = set(["Faces", "Leopards", "Motorbikes", "airplanes"])

In [ ]:
#imagedir = 'CALTECH101_ObjectCategories'
imagefilename = 'CALTECH101_ObjectCategories.zip'
import zipfile
print("[INFO] loading images...")
#imagePaths = list(paths.list_images(imagedir))
zf = zipfile.ZipFile(imagefilename)
imagelist = zf.namelist()
data = []
labels = []

In [ ]:
# loop over the image paths
#for imagePath in imagePaths:
for imagePath in imagelist:
	# extract the class label from the filename
	#print(imagePath.split(os.path.sep), os.path.sep)
	#label = imagePath.split(os.path.sep)[-2]
	label = imagePath.split('/')[-2]

	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	if label not in LABELS:
		continue

	# load the image and resize it to be a fixed 96x96 pixels,
	# ignoring aspect ratio
	imagedata = zf.read(imagePath)
	#image = cv2.imread(imagePath)
	image = cv2.imdecode(np.frombuffer(imagedata, np.uint8), 1)
	image = cv2.resize(image, (96, 96))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

In [ ]:
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
#NK: <Caveat> DO NOT CHANGE THIS PART, 25% OF DATA WILL BE USED AS TEST SET
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [ ]:
# construct the training image generator for data augmentation
#NK: You can do image augmentation further to increase your performance
#NK: Check the option of ImageDataGenerator for further processing
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [ ]:
#NK: You can change # of epochs, optimizers, and regulators
#NK: K2 regularizer Strong 0.01 ~ Week 0.0001
maxepoch = 100
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=1e-4, decay=1e-4 / maxepoch)
model = StridedNet.build(width=96, height=96, depth=3, classes=len(lb.classes_), reg=l2(0.0005))
#NK: <Caveat> DO NOT CHANGE LOSS FUNCTION, SOFTWARE CLASSFIER ALWAYS USE CATEGORICAL_CROSSTNEROPY
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
#NK: You have to check dimension of each layers before doing model.fit
#NK: Tensorflow will automatically calculate dimensional changes between layers
model.summary()

In [ ]:
#NK: 25% of test dataset is feeded into CNN model by "validation_data"
# train the network
print("[INFO] training network for {} epochs...".format(maxepoch))
H = model.fit(aug.flow(trainX, trainY, batch_size=10), validation_data=(testX, testY), steps_per_epoch=len(trainX) // 32, epochs=maxepoch, verbose=1)
#batch_size: 32->10 for small GPU memory

In [ ]:
#NK: Your performance of CNN model will be evaluated by precision, recall and f1-score
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
# plot the training loss and accuracy
import matplotlib
%matplotlib inline
plt.figure()
plt.plot(np.arange(0, maxepoch), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, maxepoch), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, maxepoch), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, maxepoch), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()